In [176]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix

%matplotlib inline

# 1. Structures of dataset

The original data set is an excel file that contains a few sheets on different categories: 
 - "Variable List"
 - "Access"
 - "Restaurants"
 - "Assistance"
 - "Insecurity"
 - "Prices_Taxes"
 - "Local","Health"
 - "Socioeconomic".

Each sheet contains food environment data (277 variables in total) for 3143 counties in the United States (rows).

### Explaining some the variable code:
- PCT: percentage
- 15/10: 2015/2010
- LACCESS: low access to store
- POP: population
- LOWI: Low income
- HHNV: Households and no car
- SNAP: SNAP households (food assistance program)
- CHILD, SENIORS: Children, seniors
- WHITE, BLACK, HISP, NHASIAN, NHNA, NHPI, MULTIR: White, black, hispanic, asian, american indian or alaska native, hawaiian or pacific islander, multiracial.

For example, 'PCT_LACCESS_HHNV15' means: Households, no car & low access to store (%), 2015

# 2. Data import and cleaning

### 2.1 Create the short list of variables 

In [177]:
# Read in the variables list 
variables=pd.read_csv('variables.csv')

In [178]:
variables.shape

(277, 8)

In [179]:
#Change column names
variables.columns=['Category','CategoryCode','SubName','VarName','VarCode','Selected','Geography','Unites']

In [180]:
variables.head()

,Category,CategoryCode,SubName,VarName,VarCode,Selected,Geography,Unites
0,Health and Physical Activity,HEALTH,Health,"Adult diabetes rate, 2008",PCT_DIABETES_ADULTS08,-1.0,CNTY10,Percent
1,Health and Physical Activity,HEALTH,Health,"Adult diabetes rate, 2013",PCT_DIABETES_ADULTS13,-1.0,CNTY10,Percent
2,Health and Physical Activity,HEALTH,Health,"Adult obesity rate, 2008",PCT_OBESE_ADULTS08,-1.0,CNTY10,Percent
3,Health and Physical Activity,HEALTH,Health,"Adult obesity rate, 2013",PCT_OBESE_ADULTS13,-1.0,CNTY10,Percent
4,Access and Proximity to Grocery Store,ACCESS,Overall,"Population, low access to store (%), 2015",PCT_LACCESS_POP15,0.0,CNTY10,Percent


In [181]:
# Variables of interests were selected and asigned numbers (-1,0,1,2) to them for filtering
short=variables.query('Selected >= -1') #select -1, 0, 1, and 2 


In [182]:
short.shape # 133 variables selected

(133, 8)

### 2.2 Create a dictionary which contains the data frames for variables in each category

In [183]:
categories=['access','assistance','insecurity',
            'local','prices','restaurants','socioeco','stores',
           'health']
files=['access.csv','assistance.csv','insecurity.csv',
       'local.csv','prices.csv','restaurants.csv','socioeco.csv','stores.csv',
      'health.csv']

In [214]:
# Create a dictionary of data frames, which contains only the average values by state for each variable
list_df={}
for category,file in zip(categories,files):
    list_df[category]=pd.read_csv(file).groupby('State').mean() # 'State' is set as the index
    print(list_df[category].shape)

(51, 42)
(51, 46)
(51, 9)
(51, 98)
(51, 9)
(51, 17)
(51, 16)
(51, 37)
(51, 12)


In [215]:
# Modify the dataframes to select fewer variables (based on the short list)
list_shortdf={}
for name in list_df.keys():
    df_short = list_df[name].copy()
    capital=name.upper()
    templist=short.loc[short['CategoryCode']==capital,'VarCode'].tolist()
    df_short = df_short[templist]

    list_shortdf[name]=df_short
    print(list_shortdf[name].shape) 


(51, 18)
(51, 22)
(51, 6)
(51, 46)
(51, 0)
(51, 8)
(51, 0)
(51, 12)
(51, 7)


In [216]:
short.head()

,Category,CategoryCode,SubName,VarName,VarCode,Selected,Geography,Unites
0,Health and Physical Activity,HEALTH,Health,"Adult diabetes rate, 2008",PCT_DIABETES_ADULTS08,-1.0,CNTY10,Percent
1,Health and Physical Activity,HEALTH,Health,"Adult diabetes rate, 2013",PCT_DIABETES_ADULTS13,-1.0,CNTY10,Percent
2,Health and Physical Activity,HEALTH,Health,"Adult obesity rate, 2008",PCT_OBESE_ADULTS08,-1.0,CNTY10,Percent
3,Health and Physical Activity,HEALTH,Health,"Adult obesity rate, 2013",PCT_OBESE_ADULTS13,-1.0,CNTY10,Percent
4,Access and Proximity to Grocery Store,ACCESS,Overall,"Population, low access to store (%), 2015",PCT_LACCESS_POP15,0.0,CNTY10,Percent


In [217]:
list_df.keys()

dict_keys(['access', 'assistance', 'insecurity', 'local', 'prices', 'restaurants', 'socioeco', 'stores', 'health'])

In [261]:
# Merge all SHROT dataframes together, to create the main dataframe for analysis: df
df=pd.concat(list_shortdf,join='outer',axis=1)
print(df.shape)

(51, 119)


In [262]:
df.head() # Multiindexing issue

access                                        \
      PCT_LACCESS_POP15 PCT_LACCESS_LOWI15 PCT_LACCESS_HHNV15   
State                                                           
AK            48.343789          18.917159          17.177076   
AL            16.171105           7.356873           4.633911   
AR            22.609819          10.931314           4.140053   
AZ            36.370275          17.289324           3.732187   
CA            19.344140           6.210381           1.544119   

                                                                    \
      PCT_LACCESS_SNAP15 PCT_LACCESS_CHILD15 PCT_LACCESS_SENIORS15   
State                                                                
AK              9.441928           12.684653              3.954663   
AL              3.103314            3.761755              2.366152   
AR              3.924008            5.170002              3.766878   
AZ              6.579101            8.963085              6.780755   
CA              1.475667            4.216773              2.868637   

                                                                  \
      PCT_LACCESS_WHITE15 PCT_LACCESS_BLACK15 PCT_LACCESS_HISP15   
State                                                              
AK              20.968158            0.500044           1.399750   
AL               9.409366            6.129109           0.488081   
AR              16.786567            4.690150           0.924264   
AZ              23.471028            0.476624           8.123239   
CA              14.753103            0.613848           4.047196   

                             ...     stores                       \
      PCT_LACCESS_NHASIAN15  ... SPECSPTH09 SNAPSPTH12 WICSPTH08   
State                        ...                                   
AK                 1.537318  ...   0.040625   1.636546  0.912355   
AL                 0.108511  ...   0.032540   1.023795  0.236434   
AR                 0.127928  ...   0.038712   0.913713  0.205605   
AZ                 0.301255  ...   0.053979   0.733253  0.165265   
CA                 0.838045  ...   0.079692   0.692391  0.154242   

                     health                                           \
      PCT_DIABETES_ADULTS08 PCT_DIABETES_ADULTS13 PCT_OBESE_ADULTS08   
State                                                                  
AK                 6.458333              7.017241          30.929167   
AL                13.055224             15.489552          33.619403   
AR                11.310667             13.308000          31.322667   
AZ                 9.413333             11.093333          26.066667   
CA                 7.606897              8.774138          24.263793   

                                                             
      PCT_OBESE_ADULTS13 PCT_HSPA15 RECFACPTH14 RECFACPTH09  
State                                                        
AK             31.368966       20.9    0.048405    0.057499  
AL             36.128358       25.4    0.047671    0.047825  
AR             35.898667       28.6    0.043767    0.050543  
AZ             28.773333       26.0    0.044966    0.060661  
CA             24.131034       25.3    0.089559    0.100914  

[5 rows x 119 columns]

In [263]:
# Drop multiindexing
df.columns = df.columns.droplevel(0)

# If need to set multiindex using 
# list_shortdf ={k: v.set_index(['State','County']) for k,v in list_shortdf.items()}

In [264]:
df.head()

,PCT_LACCESS_POP15,PCT_LACCESS_LOWI15,PCT_LACCESS_HHNV15,PCT_LACCESS_SNAP15,PCT_LACCESS_CHILD15,PCT_LACCESS_SENIORS15,PCT_LACCESS_WHITE15,PCT_LACCESS_BLACK15,PCT_LACCESS_HISP15,PCT_LACCESS_NHASIAN15,...,SPECSPTH09,SNAPSPTH12,WICSPTH08,PCT_DIABETES_ADULTS08,PCT_DIABETES_ADULTS13,PCT_OBESE_ADULTS08,PCT_OBESE_ADULTS13,PCT_HSPA15,RECFACPTH14,RECFACPTH09
State,,,,,,,,,,,,,,,,,,,,,
AK,48.343789,18.917159,17.177076,9.441928,12.684653,3.954663,20.968158,0.500044,1.399750,1.537318,...,0.040625,1.636546,0.912355,6.458333,7.017241,30.929167,31.368966,20.9,0.048405,0.057499
AL,16.171105,7.356873,4.633911,3.103314,3.761755,2.366152,9.409366,6.129109,0.488081,0.108511,...,0.032540,1.023795,0.236434,13.055224,15.489552,33.619403,36.128358,25.4,0.047671,0.047825
AR,22.609819,10.931314,4.140053,3.924008,5.170002,3.766878,16.786567,4.690150,0.924264,0.127928,...,0.038712,0.913713,0.205605,11.310667,13.308000,31.322667,35.898667,28.6,0.043767,0.050543
AZ,36.370275,17.289324,3.732187,6.579101,8.963085,6.780755,23.471028,0.476624,8.123239,0.301255,...,0.053979,0.733253,0.165265,9.413333,11.093333,26.066667,28.773333,26.0,0.044966,0.060661
CA,19.344140,6.210381,1.544119,1.475667,4.216773,2.868637,14.753103,0.613848,4.047196,0.838045,...,0.079692,0.692391,0.154242,7.606897,8.774138,24.263793,24.131034,25.3,0.089559,0.100914


In [265]:
# Save dataframe to csv
df.to_csv('atlas_bystate_selected.csv')

In [198]:
varname.head()

,VarCode,VarName
0,PCT_DIABETES_ADULTS08,"Adult diabetes rate, 2008"
2,PCT_OBESE_ADULTS08,"Adult obesity rate, 2008"
3,PCT_OBESE_ADULTS13,"Adult obesity rate, 2013"
4,PCT_LACCESS_POP15,"Population, low access to store (%), 2015"
6,PCT_LACCESS_HHNV15,"Households, no car & low access to store (%), ..."


### 2.3. Check for missing values

In [222]:
# Check for missing values
df.isna().sum().sum() # there are 47 missing values which are denoted as NaN 

# We keep those as NaN for now 

47

### 2.4.Checking for outliers 

In [224]:
# For loop to find outliers and convert them to NaN
outliers = {}

for column in df:
    data = df[column]
    mean = data.mean()
    std = data.std()
    
    # Define outliers and find their indexes
    outlier = data[(data > (mean+3*std))|(data < (mean-3*std))] # Alternatively: low,high = np.percentile(data,[1,99])
    slice_index = outlier.index # 
    
    
    # If there is no outlier in this column, go back to the beginning of the loop
    if slice_index.shape[0] == 0: continue   
    
    # Append outliers in a dictionary
    outliers[column]=df[column].loc[slice_index]
    
    #print(df[[column]].loc[slice_index],df[column].mean())
    
    # Can Convert outliers into NaN in df if needed:
    
    # data[(data > (mean+3*std))|(data < (mean-3*std))] = np.NaN
    

In [256]:
# Outliers can be accessed in the dictionary
len(outliers.values()) # There are likely 68 outliers, judging by +- 3*sd

68

In [257]:
# Can convert the outliers into a list for future reference
list_outliers=list(outliers.values())

In [259]:
list_outliers[0]

State
AK    17.177076
Name: PCT_LACCESS_HHNV15, dtype: float64